In [5]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import Analysis
import pylab
from matplotlib.gridspec import GridSpec
import SG
from scipy import interpolate
from matplotlib.colors import LinearSegmentedColormap
%matplotlib 

Using matplotlib backend: TkAgg


# Data

In [6]:
se2014 = np.load('2014se_new.npy')
se2011 = np.load('2011se_new.npy')
#data2014 = np.load('2014_Prune.npy')
data2014 = np.load('2014_Prune_new.npy')
data2011 = np.load('2011_Prune.npy')
bval2014 = [5.5845,5.5840,5.5835,5.5830,5.5825,5.5820,5.5815,5.5810,5.5805,5.5800,5.5795,5.5790,5.5785,5.5780,5.5775]
bval2011 = [5.536,5.5355,5.535,5.5345,5.534,5.5335,5.533,5.5325,5.532,5.5315,5.531,5.5305,5.53,5.5295,5.529,5.5285,5.528,5.5275,5.527,5.5265,5.526,5.5255]


# Helper Functions

In [7]:
cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 0.0, 0.0),
                   (0.5,0.0,0.0),
                   (1.0, 1.0, 1.0)),

         'blue':  ((0.0, 1.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
        
yellowredblue = LinearSegmentedColormap('YellowRedBlue', cdict)

In [8]:
def search(data, t):
    seq = data[0]    
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max < min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m
            

def clip(d,min,max):
    data = np.vstack((d[0],d[1]))
    li = search(data,min)
    ri = search(data,max)
    return data[:,li:ri+1]
    
def normalize(data):
    minv = min(data)
    maxv = max(data)
    hold = np.array(data)
    for i in range(len(hold)):
        hold[i] = (hold[i]-minv)/(maxv-minv)
    return hold

def percentile_normalize(data,p=1):
    minv = np.percentile(data,p)
    maxv = np.percentile(data,100-p)
    hold = np.array(data)
    for i in range(len(hold)):
        hold[i] = (hold[i]-minv)/(maxv-minv)
    return hold

# Plots

In [15]:
bfieldval = bval2011
data = data2011

#----------------------------------------------------------------------------
n = 1024
smoothval = 51
poly = 2
interpdim = 2
cmlabel = 'Normalized $\Delta$R$_D$ (k$\Omega$)'
data = [x for (y,x) in sorted(zip(bfieldval,data), key=lambda pair: pair[0])]
bval = sorted(bfieldval)    
xmin = None
xmax = 0
for i in data:
    if i[0][0] > xmin:
        xmin = i[0][0]
    if i[0][-1] < xmax:
        xmax = i[0][-1]
xmin += 0.5
xmax -= 0.5
xv = np.linspace(xmin,xmax,n)
intv = list()
count = 0
for i in data:
    if count == 21:
        xmin += 1
    #Try flipping filtering and interp
    dataclip = clip(i,xmin,xmax)
    dataclip[1] = SG.savitzky_golay(dataclip[1],smoothval,poly)
    tck = interpolate.splrep(dataclip[0], dataclip[1],k=1)
    y = interpolate.splev(xv,tck)
    intv.append(percentile_normalize(y,1))
    count += 1
xb = np.linspace(bval[0],bval[-1],n)
b = list()
for i in range(len(intv[0])):
    hold = list()
    for j in range(len(intv)):
        hold.append(intv[j][i])
    b.append(hold)
intb = list()
for i in b:
    tck = interpolate.splrep(bval,i,k=interpdim)
    y = interpolate.splev(xb,tck)
    intb.append(y)
Z = np.array(intb)
X,Y = pylab.meshgrid(xb,xv)


In [16]:
colortype = yellowredblue
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage (mV)')
plt.xlabel('B Field (T)')
cbar = plt.colorbar()
cbar.set_label(cmlabel)
ax.get_xaxis().get_major_formatter().set_useOffset(False)
#plt.savefig('ColorMap52_2011',dpi=1200)

In [17]:
plt.savefig('ColorMap_2011',dpi=1200)

In [81]:
se = se2011
pi54 = list()
pi = list()
for i in se[1]:
    if i[2]>1.2:
        pi54.append(i)
    else:
        pi.append(i)
pi54=np.array(pi54)
pi = np.array(pi)

In [84]:
plt.scatter((pi[:,1]),pi[:,0],s=50,color='w',edgecolor='k')
plt.scatter((pi54[:,1]),pi54[:,0],s=80,color='#15ff00',marker='*',edgecolor='k')
plt.scatter((se2011[0][:,1]),se2011[0][:,0],s=50,color='#00FFFF',marker='d',edgecolor='k')

In [85]:
plt.savefig('ColorMap52_2011',dpi=1200)

In [18]:
bfieldval = bval2014
data = data2014

#----------------------------------------------------------------------------
n = 1024
smoothval = 51
poly = 2
interpdim = 2
cmlabel = 'Normalized $\Delta$R$_D$ (k$\Omega$)'
data = [x for (y,x) in sorted(zip(bfieldval,data), key=lambda pair: pair[0])]
bval = sorted(bfieldval)    
xmin = None
xmax = 0
for i in data:
    if i[0][0] > xmin:
        xmin = i[0][0]
    if i[0][-1] < xmax:
        xmax = i[0][-1]
xmin += 0.5
xmax -= 0.5
xv = np.linspace(xmin,xmax,n)
intv = list()
count = 0
for i in data:
    if count == 21:
        xmin += 1
    #Try flipping filtering and interp
    dataclip = clip(i,xmin,xmax)
    dataclip[1] = SG.savitzky_golay(dataclip[1],smoothval,poly)
    tck = interpolate.splrep(dataclip[0], dataclip[1],k=1)
    y = interpolate.splev(xv,tck)
    intv.append(percentile_normalize(y,1))
    count += 1
xb = np.linspace(bval[0],bval[-1],n)
b = list()
for i in range(len(intv[0])):
    hold = list()
    for j in range(len(intv)):
        hold.append(intv[j][i])
    b.append(hold)
intb = list()
for i in b:
    tck = interpolate.splrep(bval,i,k=interpdim)
    y = interpolate.splev(xb,tck)
    intb.append(y)
Z = np.array(intb)
X,Y = pylab.meshgrid(xb,xv)


In [19]:
colortype = yellowredblue
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage (mV)')
plt.xlabel('B Field (T)')
cbar = plt.colorbar()
cbar.set_label(cmlabel)
ax.get_xaxis().get_major_formatter().set_useOffset(False)
#plt.savefig('ColorMap52_2011',dpi=1200)

In [20]:
plt.savefig('ColorMap_2014',dpi=1200)

In [97]:
se = se2014
pi54 = list()
pi = list()
for i in se[1]:
    if i[2]>1.2:
        pi54.append(i)
    else:
        pi.append(i)
pi54=np.array(pi54)
pi = np.array(pi)

In [98]:
plt.scatter((pi[:,1]),pi[:,0],s=50,color='w',edgecolor='k')
#plt.scatter((pi54[:,1]),pi54[:,0],s=80,color='#15ff00',marker='*',edgecolor='k')
plt.scatter((se[0][:,1]),se[0][:,0],s=50,color='#00FFFF',marker='d',edgecolor='k')

In [99]:
plt.savefig('ColorMap52_2014',dpi=1200)